<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from pandas.io.json import json_normalize

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.20.2
Pandas:  1.2.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/bikeshare.csv', header = None)
bikes.head()


# wrong:
bikes2 = pd.read_table('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/bikeshare.csv', header = 1)
bikes2.head()


# wrong:
bikes3 = pd.read_table('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/bikeshare.csv', header = 0)
bikes3.head()

,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
0,"1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,..."
1,"2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0..."
2,"3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0..."
3,"4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,..."
4,"5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,..."


?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [4]:
#ANSWER:
bikes = pd.read_table('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/bikeshare.csv', header=0, delimiter=',')
bikes.head(10000)
#using pd.read_csv much easier to read CSV files without having to format as much

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2012-02-26,1,1,2,12,0,0,0,1,0.30,0.3030,0.39,0.1343,49,256,305
9996,9997,2012-02-26,1,1,2,13,0,0,0,1,0.32,0.3333,0.36,0.0000,58,267,325
9997,9998,2012-02-26,1,1,2,14,0,0,0,1,0.34,0.3485,0.31,0.0000,68,263,331
9998,9999,2012-02-26,1,1,2,15,0,0,0,1,0.36,0.3485,0.29,0.0000,72,281,353


In [35]:
#says number of entries and column!! impt
bikes.shape

(17379, 17)

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [6]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [7]:
#usecols helps you select certain columns. so look at pandas read_excel documentation
df = pd.read_excel('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [8]:
#ANSWER
#there were no columns that agggegated data

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [9]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-06-02T23:39:01.961Z,62.4593,-148.2667,22.9,2.6,ml,NaN,NaN,NaN,0.8,...,2021-06-02T23:49:41.040Z,"74 km N of Chickaloon, Alaska",earthquake,NaN,0.4,NaN,NaN,automatic,ak,ak


In [13]:
#COMMENT: SOURCE FILE CHECKED AND HAS NO DATA BESIDES FIRST ENTRY

#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [14]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

[                         Bank NameBank           CityCity StateSt  CertCert  \
 0                    Almena State Bank             Almena      KS     15426   
 1           First City Bank of Florida  Fort Walton Beach      FL     16748   
 2                 The First State Bank      Barboursville      WV     14361   
 3                   Ericson State Bank            Ericson      NE     18265   
 4     City National Bank of New Jersey             Newark      NJ     21111   
 ..                                 ...                ...     ...       ...   
 558                 Superior Bank, FSB           Hinsdale      IL     32646   
 559                Malta National Bank              Malta      OH      6629   
 560    First Alliance Bank & Trust Co.         Manchester      NH     34264   
 561  National State Bank of Metropolis         Metropolis      IL      3815   
 562                   Bank of Honolulu           Honolulu      HI     21029   
 
                  Acquiring Institutio

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

In [15]:
##source code from austintaylor.io did not work well for me

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [16]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [4]:
#ANSWER: 17 columns and 17379 rows
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


What are the column names in `bikes`?

In [22]:
#ANSWER
bikes.columns.values.tolist()

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

What are the data types of these columns?

In [23]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [71]:
#ANSWER: it is 0 indexed so first entry will be 0. below are all the characteristics of first entry
bikes.iloc[0]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed            0.0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [24]:
#ANSWER
#method 1: shows col name, number of entries, datatype
bikes['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [25]:
#method 2: shows col name, number of entries, number of column #2 bracket is 2d array
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [7]:
#ANSWER: 2 ways
#method1:
bikes.loc[[0,1,2,3],['atemp']]

,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879


In [44]:
bikes.iloc[:4, 11]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [ ]:
#take out the # to uncomment and execute each line to see what is happening
#bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

#all created an error output

What is the correct way to access the 1st row of the DataFrame by its index?

In [11]:
#ANSWER
bikes.iloc[0, :]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed            0.0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [60]:
#ANSWER
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: Detect missing values for an array-like object. True for ever NaN value and False for a Not null value

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [150]:
bikes.isnull()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17375,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17376,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17377,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [65]:
#ANSWER: answer is none as all values shown in the df are false, indicating a value in those cells.
#used below command to doublecheck the answer
bikes.isnull().sum().sum()

0

In [ ]:
bikes.isnull().sum()

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [72]:
#ANSWER
pd.isnull(bikes).sum() > 0

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool

What is the Numpy object `nan` used for? (Write a descriptive answer.)


?
ANSWER: Marking a data point as invalid.

Answer: NaN is short for Not a number. It is used to represent entries that are undefined. It is also used for representing missing values in a dataset.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [36]:
def divide():
    try:
        #code that may raise exception
        numerator = int(input("Enter Numerator: "))
        denominator = int(input("Enter Denominator: "))
        result = numerator / denominator
        print(str("Result is: "), result)
    except:
        #code to run when exception occurs
        print("Denominator cannot be 0. Please try again")
        return "NaN"

divide()

Enter Numerator: 1
Enter Denominator: 0
Denominator cannot be 0. Please try again


'NaN'

Apply the Pandas `isna` function to the following data objects:

In [119]:
x = 2.3
y = np.nan
print(x,y)

2.3 nan


In [120]:
#ANSWER
pd.isna(x), pd.isna(y)

(False, True)

In [122]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [121]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER: 


na_values parameter is used to tell pandas they consider “[enter what you deem as NaN]” as NaN value and print NaN at the place of “[enter what you deem as NaN]”.


## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [124]:
#ANSWER: use nunique pandas.DataFrame.nunique. DataFrame.nunique(axis=0, dropna=True)[source]. 
#Count distinct observations over requested axis.
bikes['season'].nunique()

4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [132]:
#ANSWER
bikes['instant']
print (bikes.instant.idxmax() - bikes.instant.idxmin())

0


In [174]:
#range for instant col
instant = bikes["instant"]
min_instant = instant.min()
max_instant = instant.max()
range_instant = max_instant - min_instant 
print(str("instant range is: "), range_instant)
print(str("instant max is: "), max_instant)
print(str("instant min is: "),min_instant)

instant range is:  17378
instant max is:  17379
instant min is:  1


In [156]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     17379 non-null  int64         
 1   dteday      17379 non-null  datetime64[ns]
 2   season      17379 non-null  int64         
 3   yr          17379 non-null  int64         
 4   mnth        17379 non-null  int64         
 5   hr          17379 non-null  int64         
 6   holiday     17379 non-null  int64         
 7   weekday     17379 non-null  int64         
 8   workingday  17379 non-null  int64         
 9   weathersit  17379 non-null  int64         
 10  temp        17379 non-null  float64       
 11  atemp       17379 non-null  float64       
 12  hum         17379 non-null  float64       
 13  windspeed   17379 non-null  float64       
 14  casual      17379 non-null  int64         
 15  registered  17379 non-null  int64         
 16  cnt         17379 non-

In [173]:
# convert the 'dteday' column to datetime format
bikes['dteday']= pd.to_datetime(bikes['dteday'])
#check bikes.info() to see dtype again
#solve for range
dates = bikes["dteday"]
min_dates = dates.min()
max_dates = dates.max()
range_dates = max_dates - min_dates
print(str("date range is: "), range_dates)
print(str("date max is: "), max_dates)
print(str("date min is: "),min_dates)

date range is:  730 days 00:00:00
date max is:  2012-12-31 00:00:00
date min is:  2011-01-01 00:00:00


In [177]:
#range for instant col
windspeed = bikes["windspeed"]
min_windspeed = windspeed.min()
max_windspeed = windspeed.max()
range_windspeed = max_windspeed - min_windspeed
print(str("windspeed range is: "), range_windspeed)
print(str("windspeed max is: "), max_windspeed)
print(str("windspeed min is: "),min_windspeed)

windspeed range is:  0.8507
windspeed max is:  0.8507
windspeed min is:  0.0


Compute and print the overall minimum and maximum of the numeric data columns:

In [187]:
print("MIN VALUES IN THE DF ARE:")
print(bikes.min())
print("==================================")
print("MAX VALUES IN THE DF ARE:")
print(bikes.max())

MIN VALUES IN THE DF ARE:
instant                         1
dteday        2011-01-01 00:00:00
season                          1
yr                              0
mnth                            1
hr                              0
holiday                         0
weekday                         0
workingday                      0
weathersit                      1
temp                         0.02
atemp                         0.0
hum                           0.0
windspeed                     0.0
casual                          0
registered                      0
cnt                             1
dtype: object
MAX VALUES IN THE DF ARE:
instant                     17379
dteday        2012-12-31 00:00:00
season                          4
yr                              1
mnth                           12
hr                             23
holiday                         1
weekday                         6
workingday                      1
weathersit                      4
temp            

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [188]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [194]:
#ANSWER
bikes[:].quantile([.1, .25, .5, .75, .9])

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0.10,1738.8,1.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.24,0.2424,0.37,0.0000,1.0,7.0,9.0
0.25,4345.5,2.0,0.0,4.0,6.0,0.0,1.0,0.0,1.0,0.34,0.3333,0.48,0.1045,4.0,34.0,40.0
0.50,8690.0,3.0,1.0,7.0,12.0,0.0,3.0,1.0,1.0,0.50,0.4848,0.63,0.1940,17.0,115.0,142.0
0.75,13034.5,3.0,1.0,10.0,18.0,0.0,5.0,1.0,2.0,0.66,0.6212,0.78,0.2537,48.0,220.0,281.0
0.90,15641.2,4.0,1.0,11.0,21.0,0.0,6.0,1.0,2.0,0.74,0.6970,0.88,0.3582,92.0,354.0,451.2


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [197]:
type(bikes['atemp'])

pandas.core.series.Series

In [260]:
#Return a random sample of items from an axis of object.
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,category
5547,5548,2011-08-24,3,0,8,1,0,3,1,1,0.60,0.6061,0.60,0.1045,7,17,24,warm
8710,8711,2012-01-03,1,1,1,18,0,2,1,1,0.14,0.0909,0.26,0.5224,10,214,224,cool
17089,17090,2012-12-19,4,1,12,20,0,3,1,1,0.36,0.3485,0.57,0.2239,10,231,241,mild
14137,14138,2012-08-17,3,1,8,6,0,5,1,1,0.62,0.2424,0.78,0.1343,6,151,157,cool
2723,2724,2011-04-28,2,0,4,9,0,4,1,3,0.62,0.5606,0.88,0.3284,18,165,183,warm


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [212]:
#Use cut when you need to segment and sort data values into bins. This function is also useful for going from a continuous variable to a categorical variable. For example, cut could convert ages to groups of age ranges. Supports binning into an equal number of bins, or a pre-specified array of bins.
#https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.cut.html
#used for cleaning data?
"""Binning is a popular concept used while building a Regression or Logistic Model. 
Pandas package has made it easy to binning any categorical variables using the pd.cut method. 
It is simple to implement.
"""
atemp_level = pd.cut(bikes['atemp'], bins = 4)         


In [7]:
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

What is `atemp_level`?

In [234]:
#ANSWER: 
"""atemp_level has the 4 bins that observations in atemp have been divided into equidistantly.
along with showing all the observations in the range.
"""
print(atemp_level)


0        mild
1        mild
2        mild
3        mild
4        mild
         ... 
17374    mild
17375    mild
17376    mild
17377    mild
17378    mild
Name: atemp, Length: 17379, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']


Here is a random sample of `atemp_level`:

In [215]:
atemp_level.sample(5)          

15414    (0.25, 0.5]
8101     (0.25, 0.5]
6865     (0.5, 0.75]
2028     (0.25, 0.5]
2954     (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [233]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

4325    warm
5395    warm
741     cool
8761    mild
1736    mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [259]:
#ANSWER
#first turn into df
##create a new column called category, and formula for new category will be binning
bikes["category"] = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
#checked and bikes.sample(5) below shows the new column called category

"""We can use pd.cut to bin the values into ranges, then we can groupby these ranges, 
and finally call count to count the values now binned into these ranges:"""


'We can use pd.cut to bin the values into ranges, then we can groupby these ranges, \nand finally call count to count the values now binned into these ranges:'

In [261]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,category
12107,12108,2012-05-24,2,1,5,16,0,4,1,1,0.74,0.6818,0.55,0.3582,61,329,390,warm
10365,10366,2012-03-13,1,1,3,0,0,2,1,2,0.56,0.5303,0.52,0.1940,5,21,26,warm
14103,14104,2012-08-15,3,1,8,20,0,3,1,1,0.70,0.6515,0.61,0.1343,72,355,427,warm
7868,7869,2011-11-29,4,0,11,12,0,2,1,3,0.42,0.4242,0.82,0.4179,1,17,18,mild
6471,6472,2011-10-02,4,0,10,5,0,0,0,1,0.32,0.3030,0.87,0.2836,0,7,7,mild


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [39]:
#Get the number of rows and columns: df.shape
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [48]:
a.shape

(3,)

In [49]:
b.shape

(1, 3)

In [50]:
c.shape

(2, 3, 4)

In [51]:
d.shape

(4,)

In [52]:
e.shape

(2, 4)

In [0]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [4]:
#load df first and see dtypes
df_rocks = pd.read_csv('/Users/lawrenceyim/Documents/Institute of Data/Labs/EDA/data/rock.csv')
df_rocks.head()


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [4]:
#check for dtypes
df_rocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Song Clean    2230 non-null   object
 1   ARTIST CLEAN  2230 non-null   object
 2   Release Year  1653 non-null   object
 3   COMBINED      2230 non-null   object
 4   First?        2230 non-null   int64 
 5   Year?         2230 non-null   int64 
 6   PlayCount     2230 non-null   int64 
 7   F*G           2230 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


In [5]:
#check for null
df_rocks.isnull()


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2225,False,False,True,False,False,False,False,False
2226,False,False,False,False,False,False,False,False
2227,False,False,False,False,False,False,False,False
2228,False,False,False,False,False,False,False,False


In [5]:
#change column names to maniputlate
df_rocks.columns = ['song_clean','artist_clean','release_year',
                     'combined','first','year', 'playcount', 'fg']


In [6]:
#change null to 0 with replace
#df_rocks.fillna(value=0) #change to null didnt really clean the column
df_rocks['release_year'] = df_rocks['release_year'].replace(np.nan, 0)

In [ ]:
#find median to replace 0 values
df_rocks["release_year"].median()

In [8]:
#change null to 0 with replace
#df_rocks.fillna(value=0) #change to null didnt really clean the column
df_rocks['release_year'] = df_rocks['release_year'].replace(0, 1973)

In [11]:
# convert the 'Release Year' column to int
df_rocks['release_year'] = df_rocks['release_year'].apply(np.int64)


In [10]:
#noticed that songfacts.com at position 1504 is a str. will need to change to null/0 value 
df_rocks.at[1504,'release_year']= 1973

In [18]:
#CHECKED TO SEE if changed to numeric null value
display(df_rocks.iloc[1504])

song_clean                        Bullfrog Blues
artist_clean                      Rory Gallagher
release_year                                1973
combined        Bullfrog Blues by Rory Gallagher
first                                          1
year                                           1
playcount                                      1
fg                                             1
Name: 1504, dtype: object

## Check Column Names

Check column names and clean.

In [42]:
#check for dtypes
df_rocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song_clean    2230 non-null   object
 1   artist_clean  2230 non-null   object
 2   release_year  2230 non-null   int64 
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   playcount     2230 non-null   int64 
 7   fg            2230 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 139.5+ KB


In [43]:
df_rocks

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,1973,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,1973,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


In [44]:
#check for number of null
df_rocks.isnull().sum().sum()


0

In [45]:
pd.isnull(df_rocks).sum() > 0

song_clean      False
artist_clean    False
release_year    False
combined        False
first           False
year            False
playcount       False
fg              False
dtype: bool

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [ ]:
#ANSWER: COMPLETED ABOVE WHEN TRYING TO CLEAN THE DATA

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [323]:
#ANSWER: WAS CLEANED PREVIOUSLY
df_rocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song_clean    2230 non-null   object
 1   artist_clean  2230 non-null   object
 2   release_year  2230 non-null   int64 
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   playcount     2230 non-null   int64 
 7   fg            2230 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 139.5+ KB


## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [48]:
#fixed to change out erroneous outlier
print("MIN VALUES IN THE DF ARE:")
print(df_rocks.min())
print("==================================")
print("MAX VALUES IN THE DF ARE:")
print(df_rocks.max())

MIN VALUES IN THE DF ARE:
song_clean                     #9 Dream
artist_clean                .38 Special
release_year                       1955
combined        #9 Dream by John Lennon
first                                 1
year                                  0
playcount                             0
fg                                    0
dtype: object
MAX VALUES IN THE DF ARE:
song_clean                     Ziggy Stardust
artist_clean                             a-ha
release_year                             2014
combined        Ziggy Stardust by David Bowie
first                                       1
year                                        1
playcount                                 142
fg                                        142
dtype: object


In [47]:
#change release_year
df_rocks['release_year'] = df_rocks['release_year'].replace(1071, 1973)

In [49]:
df_rocks.describe()

,release_year,first,year,playcount,fg
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1977.123318,1.0,0.741256,16.872646,15.048430
std,8.373972,0.0,0.438043,25.302972,25.288366
min,1955.000000,1.0,0.000000,0.000000,0.000000
25%,1973.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000


## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [16]:

#answer here
def era_finder():
    df_rocks['post_70s'] = ["post 70s times" if x > 1970 else "super old" for x in df_rocks['release_year']]
    x = int(input("Enter Row Number: "))
    return df_rocks.iloc[[x],[0,1,2,8]]

In [18]:
#testing it out. it works!
era_finder()

Enter Row Number: 1200


,song_clean,artist_clean,release_year,post_70s
1200,On a Plain,Nirvana,1991,post 70s times


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [12]:
#test on sub-df "song_era"
def floater():
    df_rocks2 = df_rocks.apply(pd.to_numeric, errors='coerce', downcast='float')
    return df_rocks2


In [13]:
floater()

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0
1,NaN,NaN,1973.0,NaN,1.0,0.0,3.0,0.0
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,1973.0,NaN,1.0,0.0,1.0,0.0
2226,NaN,NaN,1981.0,NaN,1.0,1.0,32.0,32.0
2227,NaN,NaN,1975.0,NaN,1.0,1.0,109.0,109.0
2228,NaN,NaN,1983.0,NaN,1.0,1.0,1.0,1.0


In [15]:
df_rocks2 = df_rocks.apply(pd.to_numeric, errors='coerce', downcast='float')

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

In [32]:
#IT WORKS! created a seperate df_rocks2 df at a different time besides the function. it doesnt save from the function
df_rocks2

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0
1,NaN,NaN,1973.0,NaN,1.0,0.0,3.0,0.0
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,1973.0,NaN,1.0,0.0,1.0,0.0
2226,NaN,NaN,1981.0,NaN,1.0,1.0,32.0,32.0
2227,NaN,NaN,1975.0,NaN,1.0,1.0,109.0,109.0
2228,NaN,NaN,1983.0,NaN,1.0,1.0,1.0,1.0


In [16]:
df_rocks2.describe()

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
count,2.000000,0.0,2230.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1976.718872,NaN,1.0,0.741256,16.872646,15.048430
std,1367.544556,NaN,20.935745,NaN,0.0,0.438043,25.302971,25.288366
min,45.000000,NaN,1071.000000,NaN,1.0,0.000000,0.000000,0.000000
25%,528.500000,NaN,1973.000000,NaN,1.0,0.000000,1.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000


>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



